<img src="https://github.com/CropEvol/lecture/blob/master/ILAS_2020/01/images/banner.png?raw=true" alt="ILASセミナー「ゲノム博物学入門」" height="100px" align="middle">

# データ解析入門 - ゲノム解析入門 -


## Introduction

　今回は、ゲノムデータ解析をおこないます。**MutMap** (Abe et al., *Nat. Biotechnol.*, 2012) と呼ばれる解析手法を使って、イネの葉が「淡緑色（pale green）」となる原因遺伝子を探します。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/mutmap04.jpg?raw=true" alt="mutmap04" height="230px">

　MutMap は、突然変異形質をもつ複数の個体のDNAをひとつにまとめてシーケンシングし、そのシーケンスのデータ解析をおこない、形質に関わる遺伝子がゲノム上のどこにあるかを探索する解析手法（遺伝子マッピング手法）の一つです。

次のような手順で解析を行います。

1. ある系統（オリジナル系統）から突然変異系統を作出します。
1. 突然変異系統とオリジナル系統を交配します。その交配後第二世代（F2集団）では、オリジナル系統と同じ形質を持った個体と突然変異形質を持った個体が分離してきます。
1. 突然変異形質をもった個体のDNAを集めます（以下バルクDNA）。
1. オリジナル系統のDNAとバルクDNAをシーケンスします。
1. 両者のシーケンスを比較すると、突然変異の位置と、オリジナル系統の塩基や突然変異の塩基の種類がわかります。また、それぞれの塩基がいくつあるか（オリジナル系統と同じ塩基の個数と突然変異系統と同じ塩基の個数）といった情報も得られます。
1. 突然変異形質を持つF2個体は、形質に関わる何らかの変異を共通で持っているはずです。そのため、形質に関わる変異箇所では、突然変異系統の塩基のみ検出されることが期待できます。そこで、それぞれの一塩基多型（**SNP**）で、突然変異塩基の割合（**SNP-index**と呼ばれるもの）を調べます。
1. ゲノム全体にわたってSNP-index調べて、形質の原因となる遺伝子領域を探します。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/mutmap01.jpg?raw=true" alt="mutmap01" height="400px">

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/mutmap02.png?raw=true" alt="mutmap02" height="200px">

　MutMapについてより詳しく知りたい方は、下記の論文をご覧ください。

> Abe, A., Kosugi, S., Yoshida, K., Natsume, S., Takagi, H., Kanzaki, H., Matsumura, H., Yoshida, K., Mitsuoka, C., Tamiru, M., Innan, H., Cano, L., Kamoun, S., Terauchi, R. (2012). [Genome sequencing reveals agronomically important loci in rice using MutMap.](https://www.nature.com/articles/nbt.2095)  Nature biotechnology, 30(2), 174.

　このテキストでは、下記のような、イネ第10染色体のデータのみを扱います。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/mutmap03.jpg?raw=true" alt="mutmap03" height="350px">

　実習を始める前に、サンプルデータのダウンロードと、Python (pandas)での読み込みをおこなってください。

In [0]:
"""
＊重要＊
最初にこのセルを実行してください。サンプルファイルをダウンロードします。
"""
!wget -O mutmap_bulk.txt https://raw.githubusercontent.com/CropEvol/lecture/master/data/mutmap_bulk.txt

In [0]:
import pandas as pd   # pandasを準備する 

# ファイルを読み込む
# header=None（列名行なし）により、列名として通し番号が付けられます。
# names=[リスト]で、通し番号の代わりに列名を付けられます。
df = pd.read_csv('mutmap_bulk.txt', sep="\t", 
                 header=None, names=['chr', 'pos', 'ref_nucl', 'alt_nucl', 'ref_N', 'alt_N'])

# 表示
df

## 今回の実習内容

1. SNP-indexを求める
1. 条件にあうデータを抽出する
1. SNP-indexのグラフを描く
1. Sliding window解析（発展）


##1. SNP-indexを求める

　SNP-indexを算出してみましょう。計算に使う列を取り出し、次の計算をおこなうだけです。
```
# SNP-index計算式
SNP-index = alt_N / (ref_N + alt_N)
```



In [0]:
# 分母: ref_N + alt_N
df["ref_N"] + df["alt_N"]

### 実習1

　SNP-indexの計算式を記述して、データフレームに新しい列 `snp_index` を追加してください。

In [0]:
df['snp_index'] = 

df

#### 解答例

In [0]:
df['snp_index'] = df['alt_N'] / (df['ref_N'] +df['alt_N'])

# 表示
df

## 2. 条件に合うデータを抽出する

　前回のテキストで、条件に合うデータの抽出方法を学習しました。おさらいですが、コードの書き方は以下の通りです。

```python
データフレーム[ (条件式)　]   # 条件式が一つの場合、丸括弧()は省略可能です

データフレーム[ (条件式1) & (条件式2)　]   # 条件2つ（AND条件）
データフレーム[ (条件式1) | (条件式2)　]   # 条件2つ（OR条件）
```
　確認として例を見てみましょう。次のセルを実行してください。


In [0]:
df['ref_nucl']=='G'         # True/Falseリストが得られる
#df[ df['ref_nucl']=='G' ]  # Trueのデータ行が得られる

### 実習2

　SNP-indexが0.8以上のデータ行を取り出してください。

#### 解答例

In [0]:
df[df['snp_index'] >= 0.8]

## 3. SNP-indexのグラフを描く

　Pythonのグラフ描画で有名なライブラリ **Matplotlib** を使ってグラフを描いてみましょう。Matplotlibの基本は、前回のテキストで勉強したとおり、「透明なシート（レイヤー）を重ねて、ひとつのグラフを作っていく」です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/drawing_graph.png?raw=true" alt="basis_of_matplotlib_drawing" height="200px">

　ここでは、x軸に染色体上の位置、y軸にSNP-indexをとったグラフ（散布図）を作成してみましょう。


In [0]:
import matplotlib.pyplot as plt   # ライブラリを読み込む

df['snp_index'] = df['alt_N'] / (df['ref_N'] + df['alt_N'])  

x = df['pos']         # x軸データ
y = df['snp_index']   # y軸データ

fig = plt.figure(figsize=[12,4])                         # グラフの大きさを設定（省略可）
#plt.scatter(x, y, color='gray')                          # 散布図
#plt.title('SNP-index on chromosome 10', fontsize=24)  # グラフタイトル
#plt.xlabel('Position (x 10 Mb)', fontsize=16)  # x軸ラベル
#plt.ylabel('SNP-index', fontsize=16)           # y軸ラベル
plt.show()                                    # グラフ表示（省略可）

### 実習3

　全SNP-indexの散布図のうち、SNP-indexが0.8以上の点を赤色にしてください。


In [0]:
import matplotlib.pyplot as plt  

df['snp_index'] = df['alt_N'] / (df['ref_N'] + df['alt_N'])  

x = df['pos']        
y = df['snp_index']   

fig = plt.figure(figsize=[12,4])

# 全SNP-indexを表示
plt.scatter(x, y, color='gray') 
plt.title('SNP-index on chromosome 10', fontsize=24)
plt.xlabel('Position (x 10 Mb)', fontsize=16)
plt.ylabel('SNP-index', fontsize=16)           

# SNP-indexが0.8以上大きいものを赤色で表示
# （上で作成した散布図に、赤色の散布図を重ねる）

# 元のデータフレームdfを上書きしないように、別名のデータフレームを作成
df1 = df[ df['snp_index'] >= 0.8 ]   

x1 = df1[]   # x軸データを[]内に書き足す
y1 = df1[]   # y軸データを[]内に書き足す
plt.scatter()   # 赤色で表示させるためのコードを()内に書き足す

plt.show()      # グラフ表示

#### 解答例

In [0]:
import matplotlib.pyplot as plt  

df['snp_index'] = df['alt_N'] / (df['ref_N'] + df['alt_N'])  

x = df['pos']        
y = df['snp_index']   

fig = plt.figure(figsize=[12,4])    
plt.scatter(x, y, color='gray') 
plt.title('SNP-index on chromosome 10', fontsize=24)
plt.xlabel('Position (x 10 Mb)', fontsize=16)
plt.ylabel('SNP-index', fontsize=16)           

# SNP-indexが0.8より大きいものを表示
# "df"だとオリジナルのデータフレームが上書きされてしまうため、別名のデータフレームを作成
df1 = df[ df['snp_index'] >= 0.8 ]   

x1 = df1["pos"]   # x軸データを[]内に書き足す
y1 = df1["snp_index"]   # y軸データを[]内に書き足す
plt.scatter(x1, y1, color='red')   # 赤色で表示させるためのコードを()内に書き足す

plt.show()      # グラフ表示

## 4. Sliding Window解析（発展）

　MutMap解析では、SNP-indexの区間平均をゲノムに全体にわたって調べる解析（**Sliding Window解析**）というのをおこない、形質にかかわるゲノム領域を特定します。

　形質にかかわる領域付近では、直接の原因となる塩基座位だけでなく、周囲の塩基座位のSNP-indexも1に近づきます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/sliding_window_ja.jpg?raw=true" alt="sliding_window" height="400px">

　ここでは上の図のように、Chromosome 10上のSNP-indexの推移を調べてみましょう。Sliding window解析をおこなうプログラムはすでに書かれており、実行するだけでOKです。

### 解析方法

　SNP-indexのSliding Window解析は次のような手順で解析します。

    1.区間サイズとステップサイズを決める。
    2.各区間の代表値（ここでは区間の中央値）用のリストと、区間内の平均SNP-index用のリストを用意する。
    3.while構文（for構文とは別のループ処理をおこなう構文）を使って全ての区間を調べていく。
        区間内に含まれるデータを抽出する。
        抽出したデータからSNP-indexの平均値を算出する。
        区間の中央値と算出した平均SNP-indexを、それぞれのリストに追加する。
        ゲノムの全長を全て調べたらwhileから出る
    4.x軸に中央値、y軸に平均SNP-indexの値をとったグラフ（折れ線グラフなど）を作成する。


In [0]:
###### Sliding Window解析 ######
#---  NumPyライブラリの読み込み ---
import numpy as np

#--- 区間サイズとステップサイズ ---
CHROM_SIZE = 23207287       # Chromosome 10　の全長 (bp)
WIN_SIZE       = 1 * 1000 * 1000     #  1 Mb = 1000 kb = 1,000,000 bp
STEP_SIZE     = 0.2 * 1000* 1000     #  0.2 Mb = 200 kb = 200,000 bp

#--- 区間データ用のリストを準備 ---
win_position  = []  # 区間中央値用リスト
win_snpindex = []  # 平均SNP-index用リスト

#--- 全ての区間を調べる ---
"""
/// 調べる区間 ///
start, end
0, 0+1000 (kb)
200, 200+1000
400, 400+1000
  :
  :

/// WIN_SIZEとSTEP_SIZEを使って表現すると... ///
繰り返し数:　n = 0, 1, 2, ...

start = STEP_SIZE * n  
end = start + WIN_SIZE


end > CHROM_SIZEになったとき、繰り返しを終える。

このように、繰り返し数を指定しない場合は、for文ではなく、while文を使用します。
"""

n = 0 # 繰り返し数
while True:
    
    #--- 区間のstartとend position ---
    start = STEP_SIZE * n 
    end   = start + WIN_SIZE
    
    #--- 区間の中央値をリストに追加する ---
    p = (start + end) / 2
    win_position.append(p)
    
    #--- 区間内データを抽出 ---
    sub = df[(df['pos'] >= start) & (df['pos'] < end)]
    
    #--- SNP-indexの平均値を算出 ---
    i = sub['snp_index'].mean()
    win_snpindex.append(i)
        
    #--- 繰り返し数を+1 ---
    n += 1
    
    #--- 全ての区間を調べた時、whileから出る ---
    if end > CHROM_SIZE:
        break

#--- グラフ作成  ---
fig = plt.figure(figsize=[12,4])
plt.scatter(x, y, color='gray')      # 全データ
plt.title('SNP-index on chromosome 10', fontsize=24)  # グラフタイトル
plt.xlabel('Position (x 10 Mb)', fontsize=16)  # x軸ラベル
plt.ylabel('SNP-index', fontsize=16)        # y軸ラベル

#--- 平均SNP-indexの折れ線グラフ（重ね描き） ---
plt.plot(win_position, win_snpindex, color='blue')

　平均SNP-indexが高いゲノム領域（22,482,910bpの辺り）には、「Chlorophyllide a oxygenase (Os10t0567400)」という遺伝子が存在しています。この遺伝子のノックアウト形質転換体（遺伝子の機能を壊した個体）は、葉の色が淡緑色になることが明らかになっています([Abe et al., 2012](https://www.nature.com/articles/nbt.2095))。

---

## まとめ

　ゲノム解析の一例として、**MutMap解析**をおこないました。**pandas**を使って、列同士の数値計算（**SNP-index**の計算）をおこないました。また、**Matplotlib**で、データをグラフ化しました。やや複雑なプログラムにはなりますが、ゲノム全体に渡って区間平均を求めるようなゲノム解析（**Sliding window解析**）をおこなえることを学びました。

　今回扱ったデータは、272行の小規模なデータ（イネ第10染色体のみのデータ）でしたが、より大規模なデータ（数万行、数10万行のデータ）も同様に解析可能です。



---

## 練習問題
　第12染色体のデータを使って、SNP-indexの計算、グラフ描画をおこなってください。

In [0]:
"""
＊重要＊
このセルを実行してください。練習問題用のサンプルファイルをダウンロードします。
"""
!wget -O mutmap_bulk_chr12.txt https://raw.githubusercontent.com/CropEvol/lecture/master/data/mutmap_for_hw.txt

In [0]:
import pandas as pd   # pandasを準備する 

# ファイルを読み込む
# header=None（列名行なし）により、列名として通し番号が付けられます。
# names=[リスト]で、通し番号の代わりに列名を付けられます。
df = pd.read_csv('mutmap_bulk_chr12.txt', sep="\t", 
                 header=None, names=['chr', 'pos', 'ref_nucl', 'alt_nucl', 'ref_N', 'alt_N'])

# 表示
df

### 練習問題1. SNP-indexを求める

### 練習問題2. SNP-indexのグラフを描く

### 練習問題3. SNP-indexのSliding window解析をおこなう
イネ第12染色体の全長(bp) = 27,531,856